In [1]:
from PIL import Image

def change_red_to_blue(input_image_path, output_image_path):
    # Open an image file
    with Image.open(input_image_path) as img:
        # Convert image to RGBA format if not already in this mode
        img = img.convert("RGBA")
        # Get pixel data
        pixels = img.load()

        # Iterate through pixels
        for y in range(img.height):
            for x in range(img.width):
                r, g, b, a = pixels[x, y]  # Get RGBA values for the pixel
                
                # If the pixel is red (r > 150 and g < 100 and b < 100), change it to blue
                if r > 150 and g < 100 and b < 100:
                    # Change the color to blue (0, 0, 255)
                    pixels[x, y] = (0, 0, 255, a)

        # Save the modified image
        img.save(output_image_path)

# Example usage
input_image_path = r"E:\EEP\back.jpg"
output_image_path = r"E:\EEP\output_image.png"
change_red_to_blue(input_image_path, output_image_path)


In [46]:
import pandas as pd
from datetime import datetime

# List of dictionaries
list1 = [{'Sno': 1, 'Date': '12JUL2024', 'AMT': 21},
         {'Sno': 2, 'Date': '13SEP2024', 'AMT': -22},
         {'Sno': 3, 'Date': '24SEP2024', 'AMT': 35}]

# Convert the list to a DataFrame
df = pd.DataFrame(list1)

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d%b%Y')

# Step 1: Drop duplicates or aggregate the data to ensure unique dates
# We keep the last entry for each day (group by 'Date' and take the last entry)
df_unique = df.groupby('Date').last()

# Step 2: Resample the data to ensure every date is accounted for, and fill missing dates with the previous day's data
# This assumes daily data, so we use 'D' for resampling by day
df_resampled = df_unique.resample('D').ffill()

# Step 3: Reset the index if necessary
df_resampled.reset_index(inplace=True)

# Display the result
print(df_resampled)

# You can write the result to an Excel file if needed
df_resampled.to_excel('filtered_output.xlsx', index=False)


         Date  Sno  AMT
0  2024-07-12    1   21
1  2024-07-13    1   21
2  2024-07-14    1   21
3  2024-07-15    1   21
4  2024-07-16    1   21
..        ...  ...  ...
70 2024-09-20    2  -22
71 2024-09-21    2  -22
72 2024-09-22    2  -22
73 2024-09-23    2  -22
74 2024-09-24    3   35

[75 rows x 3 columns]


In [47]:
Range1=['01AUG2024','15AUG2024']
Range2=['16AUG2024','30AUG2024']
Range3=['01SEP2024','20SEP2024']



In [48]:
acc=0
acc_amt = 0
cut=0
now=0
data=df_resampled
for i in range(len(data)):
    if data.at[i,'Date'] >= datetime.strptime(Range3[0],'%d%b%Y') and data.at[i,'Date'] <= datetime.strptime(Range3[1],'%d%b%Y') :
        acc = acc + 1
        if data.at[i,"AMT"] < 0:
            acc_amt+=1
            
    

In [44]:
acc_amt

8

str

In [81]:
import tkinter as tk
from tkinter import messagebox
from datetime import datetime , timedelta
import pandas as pd
import random

dfs=pd.DataFrame()
df=pd.DataFrame()
# Simulated function to get data from an external source (you can replace this with your actual data-fetching logic)
def get_data():
    start_date = datetime.now() - timedelta(days=29)
    
    data = {
        "Date": [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(30)],
        "Amount": [random.uniform(100.00, 1000.00) for _ in range(25)] + [random.uniform(-1000.00, -100.00) for _ in range(5)]
    }

    # Shuffle the amounts so that negative values are randomly placed
    random.shuffle(data['Amount'])

    df = pd.DataFrame(data)
    return df

# Function to calculate and display total days and other values, and display data as a table
# In calculate_days function
def filter_data():
    global dfs
    df=dfs
    if debit_var.get() == 1 and credit_var.get() == 0:
            # Show only negative amounts (debits)
        df = dfs[dfs['Amount'] < 0]
    elif credit_var.get() == 1 and debit_var.get() == 0:
        # Show only positive amounts (credits)
        df = dfs[dfs['Amount'] > 0]
    
    # Populate "Total Number of Entries" with the number of filtered entries in the DataFrame
    total_days_var.set(f"Total Number of Entries: {len(df)}")

    # Clear previous data from the display area
    display_area.delete(1.0, tk.END)

    
    
    # Display table headers
    display_area.insert(tk.END, "\nDate\t\tAmount\n")
    display_area.insert(tk.END, "-------------------------\n")

    # Display filtered DataFrame content in the table format
    for index, row in df.iterrows():
        display_area.insert(tk.END, f"{row['Date']}\t{row['Amount']:.2f}\n")


def calculate_days():
    global dfs
    try:
        tran_date = datetime.strptime(tran_date_entry.get(), "%Y-%m-%d")
        settlement_date = datetime.strptime(settlement_date_entry.get(), "%Y-%m-%d")
        filter_start = datetime.strptime(filter_start_entry.get(), "%Y-%m-%d")
        filter_end = datetime.strptime(filter_end_entry.get(), "%Y-%m-%d")

        # Get data from external source (simulated here with get_data function)
        dfs = get_data()
        df=dfs
        # Filter DataFrame based on checkbox selections
        filter_data()

    except ValueError as e:
        messagebox.showerror("Invalid Date", "Please enter dates in the format YYYY-MM-DD")


# Main window
root = tk.Tk()
root.title("Transaction Information")

# Labels and Entries for Account and Branch
tk.Label(root, text="Account:").grid(row=0, column=0, padx=10, pady=5)
account_entry = tk.Entry(root)
account_entry.grid(row=0, column=1, padx=10, pady=5)

tk.Label(root, text="Branch:").grid(row=0, column=2, padx=10, pady=5)
branch_entry = tk.Entry(root)
branch_entry.grid(row=0, column=3, padx=10, pady=5)

# Labels and Entries for TranDate and SettlementDate
tk.Label(root, text="TranDate (YYYY-MM-DD):").grid(row=1, column=0, padx=10, pady=5)
tran_date_entry = tk.Entry(root)
tran_date_entry.grid(row=1, column=1, padx=10, pady=5)

tk.Label(root, text="SettlementDate (YYYY-MM-DD):").grid(row=1, column=2, padx=10, pady=5)
settlement_date_entry = tk.Entry(root)
settlement_date_entry.grid(row=1, column=3, padx=10, pady=5)

# Labels and Entries for FilterStart and FilterEnd
tk.Label(root, text="FilterStart (YYYY-MM-DD):").grid(row=2, column=0, padx=10, pady=5)
filter_start_entry = tk.Entry(root)
filter_start_entry.grid(row=2, column=1, padx=10, pady=5)

tk.Label(root, text="FilterEnd (YYYY-MM-DD):").grid(row=2, column=2, padx=10, pady=5)
filter_end_entry = tk.Entry(root)
filter_end_entry.grid(row=2, column=3, padx=10, pady=5)

# Checkboxes for Debit and Credit
debit_var = tk.IntVar()
credit_var = tk.IntVar()

tk.Checkbutton(root, text="Debit", variable=debit_var,command=filter_data).grid(row=3, column=0, padx=10, pady=5)
tk.Checkbutton(root, text="Credit", variable=credit_var,command=filter_data).grid(row=3, column=1, padx=10, pady=5)

# Label for Total Number of Days (non-editable)
total_days_var = tk.StringVar()
total_days_var.set("Total Number of Days: __")
total_days_label = tk.Label(root, textvariable=total_days_var, relief="sunken")
total_days_label.grid(row=4, column=0, columnspan=4, padx=10, pady=5)

# Display area for Date, Seq, and Amt
tk.Label(root, text="Display Area:").grid(row=6, column=0, padx=10, pady=5)

display_area = tk.Text(root, height=10, width=50)
display_area.grid(row=7, column=0, columnspan=4, padx=10, pady=5)

# Button to calculate total days and display information
tk.Button(root, text="Submit", command=calculate_days).grid(row=5, column=0, columnspan=4, padx=10, pady=5)

# Run the application
root.mainloop()


In [1]:
import tkinter as tk

# Create main window
root = tk.Tk()
root.title("Radio Button Example")

# Variable to store the value of the selected radio button
selected_option = tk.StringVar()

# Function to display the selected option
def show_selection():
    print(f"Selected option: {selected_option.get()}")

# Create radio buttons
radio1 = tk.Radiobutton(root, text="Option 1", variable=selected_option, value="Option 1", command=show_selection)
radio2 = tk.Radiobutton(root, text="Option 2", variable=selected_option, value="Option 2", command=show_selection)
radio3 = tk.Radiobutton(root, text="Option 3", variable=selected_option, value="Option 3", command=show_selection)

# Set a default value
selected_option.set("Option 1")

# Pack the radio buttons
radio1.pack()
radio2.pack()
radio3.pack()

# Run the GUI event loop
root.mainloop()


Selected option: Option 2
Selected option: Option 1
Selected option: Option 3
Selected option: Option 1
Selected option: Option 2
Selected option: Option 3


In [ ]:
!pip 

In [3]:
import os 
import openai
API_KEY =os.getenv("OPEN_API_KEY")

ModuleNotFoundError: No module named 'openai'

In [ ]:
messages = [
    {"role": "system", "content": "You are a kind helpful assistant."},
]